In [1]:
import psi4
from chargecraft.storage.data_classes import ESPSettings, PCMSettings, DDXSettings
from chargecraft.storage.qcarchive_transfer import QCArchiveToLocalDB
from qcportal import PortalClient
from chargecraft.storage.storage import MoleculePropRecord, MoleculePropStore
from openff.recharge.grids import LatticeGridSettings
from concurrent.futures import ProcessPoolExecutor, as_completed
from openff.toolkit.topology import Molecule
from openff.recharge.grids import GridGenerator, GridSettingsType
from openff.recharge.esp import DFTGridSettings
from multiprocessing import get_context
from tqdm import tqdm
from qcportal.singlepoint.record_models import SinglepointRecord
from typing import Union
from qcelemental.models import Molecule as QCMolecule
from openff.recharge.esp.qcresults import reconstruct_density, compute_esp
from openff.units import unit
from chargecraft.storage.data_classes import ESPSettings, PCMSettings, DDXSettings
import numpy as np
from openff.recharge.esp.qcresults import reconstruct_density
from typing import Optional

The OpenEye Toolkits are found to be installed but not licensed and therefore will not be used.
The OpenEye Toolkits require a (free for academics) license, see https://docs.eyesopen.com/toolkits/python/quickstart-python/license.html
The OpenEye Toolkits are found to be installed but not licensed and therefore will not be used.
The OpenEye Toolkits require a (free for academics) license, see https://docs.eyesopen.com/toolkits/python/quickstart-python/license.html


In [2]:
def dft_grid_settings(item: SinglepointRecord) -> DFTGridSettings | None:
    if 'xc grid radial points' not in item.properties:
        return DFTGridSettings.Default
    if item.properties['xc grid radial points'] == 75.0 and item.properties['xc grid spherical points'] == 302.0:
        return DFTGridSettings.Default
    elif item.properties['xc grid radial points'] == 85.0 and item.properties['xc grid spherical points'] == 434.0:
        return DFTGridSettings.Medium
    elif item.properties['xc grid radial points'] == 99.0 and item.properties['xc grid spherical points'] == 590.0:
        return DFTGridSettings.Fine
    else:
        return None

In [3]:
client = PortalClient("api.qcarchive.molssi.org")


In [4]:
grid_settings = LatticeGridSettings(
    type="fcc", spacing=0.5, inner_vdw_scale=1.4, outer_vdw_scale=2.0
)
molecules = [record for record in client.query_records(dataset_id=347)]
# Ensure orientation is correct
item = molecules[0]
qc_mol = item.molecule
qc_data = qc_mol.dict()
qc_data['fix_com'] = True
qc_data['fix_orientation'] = True
qc_mol = QCMolecule.from_data(qc_data)

openff_molecule = Molecule.from_qcschema(qc_mol, allow_undefined_stereo=True)
openff_conformer = openff_molecule.conformers[0]
esp_settings = ESPSettings(
    basis=item.specification.basis,
    method=item.specification.method,
    grid_settings=grid_settings,
    pcm_settings=PCMSettings(
        solver='',
        solvent='',
        radii_model='',
        radii_scaling='',
        cavity_area=''
    ) if 'PCM' in item.specification.keywords else None,
    ddx_settings=DDXSettings(
        solvent=None if 'ddx_solvent_epsilon' not in item.specification.keywords
        else item.specification.keywords['ddx_solvent'],
        epsilon=item.specification.keywords.get('ddx_solvent_epsilon', None),
        radii_set='uff',
        ddx_model=item.specification.keywords.get('ddx_model', '').upper()
    ) if 'ddx' in item.specification.keywords else None,
    psi4_dft_grid_settings=dft_grid_settings(item)
)

In [5]:
def compute_properties(qc_molecule: "qcelemental.models.Molecule",
                       density: np.ndarray,
                       esp_settings: ESPSettings) -> dict[str, np.ndarray]:
    psi4.core.be_quiet()

    psi4_molecule = psi4.geometry(qc_molecule.to_string("psi4", "angstrom"))
    psi4_molecule.reset_point_group("c1")

    psi4_wavefunction = psi4.core.RHF(
        psi4.core.Wavefunction.build(psi4_molecule, esp_settings.basis),
        psi4.core.SuperFunctional(),
    )
    psi4_wavefunction.Da().copy(psi4.core.Matrix.from_array(density))

    psi4.oeprop(
        psi4_wavefunction,
        "DIPOLE",
        "QUADRUPOLE",
        "MULLIKEN_CHARGES",
        "LOWDIN_CHARGES",
        "MBIS_CHARGES",
        "MBIS_DIPOLE",
        "MBIS_QUADRUPOLE"
    )

    variables_dictionary = dict()
    variables_dictionary["MULLIKEN_CHARGES"] = psi4_wavefunction.variable("MULLIKEN_CHARGES") * unit.e
    variables_dictionary["LOWDIN_CHARGES"] = psi4_wavefunction.variable("LOWDIN_CHARGES") * unit.e
    variables_dictionary["MBIS CHARGES"] = psi4_wavefunction.variable("MBIS CHARGES") * unit.e
    variables_dictionary["MBIS DIPOLE"] = psi4_wavefunction.variable("MBIS DIPOLES") * unit.e * unit.bohr_radius
    variables_dictionary["MBIS QUADRUPOLE"] = psi4_wavefunction.variable("MBIS QUADRUPOLES") * unit.e * unit.bohr_radius**2
    variables_dictionary["MBIS OCTOPOLE"] = psi4_wavefunction.variable("MBIS OCTUPOLES") * unit.e * unit.bohr_radius**3
    variables_dictionary["DIPOLE"] = psi4_wavefunction.variable("DIPOLE") * unit.e * unit.bohr_radius
    variables_dictionary["QUADRUPOLE"] = psi4_wavefunction.variable("QUADRUPOLE") * unit.e * unit.bohr_radius**2
    variables_dictionary["ALPHA_DENSITY"] = psi4_wavefunction.Da().to_array()
    variables_dictionary["BETA_DENSITY"] = psi4_wavefunction.Db().to_array()

    return variables_dictionary

In [6]:
def compute_properties_bohr(qc_molecule: "qcelemental.models.Molecule",
                       density: np.ndarray,
                       esp_settings: ESPSettings) -> dict[str, np.ndarray]:
    psi4.core.be_quiet()

    psi4_molecule = psi4.geometry(qc_molecule.to_string("psi4", "bohr"))
    psi4_molecule.reset_point_group("c1")

    psi4_wavefunction = psi4.core.RHF(
        psi4.core.Wavefunction.build(psi4_molecule, esp_settings.basis),
        psi4.core.SuperFunctional(),
    )
    psi4_wavefunction.Da().copy(psi4.core.Matrix.from_array(density))

    psi4.oeprop(
        psi4_wavefunction,
        "DIPOLE",
        "QUADRUPOLE",
        "MULLIKEN_CHARGES",
        "LOWDIN_CHARGES",
        "MBIS_CHARGES",
        "MBIS_DIPOLE",
        "MBIS_QUADRUPOLE"
    )

    variables_dictionary = dict()
    variables_dictionary["MULLIKEN_CHARGES"] = psi4_wavefunction.variable("MULLIKEN_CHARGES") * unit.e
    variables_dictionary["LOWDIN_CHARGES"] = psi4_wavefunction.variable("LOWDIN_CHARGES") * unit.e
    variables_dictionary["MBIS CHARGES"] = psi4_wavefunction.variable("MBIS CHARGES") * unit.e
    variables_dictionary["MBIS DIPOLE"] = psi4_wavefunction.variable("MBIS DIPOLES") * unit.e * unit.bohr_radius
    variables_dictionary["MBIS QUADRUPOLE"] = psi4_wavefunction.variable("MBIS QUADRUPOLES") * unit.e * unit.bohr_radius**2
    variables_dictionary["MBIS OCTOPOLE"] = psi4_wavefunction.variable("MBIS OCTUPOLES") * unit.e * unit.bohr_radius**3
    variables_dictionary["DIPOLE"] = psi4_wavefunction.variable("DIPOLE") * unit.e * unit.bohr_radius
    variables_dictionary["QUADRUPOLE"] = psi4_wavefunction.variable("QUADRUPOLE") * unit.e * unit.bohr_radius**2
    variables_dictionary["ALPHA_DENSITY"] = psi4_wavefunction.Da().to_array()
    variables_dictionary["BETA_DENSITY"] = psi4_wavefunction.Db().to_array()

    return variables_dictionary

In [7]:
density = reconstruct_density(wavefunction=item.wavefunction, n_alpha=item.properties['calcinfo_nalpha'])
variables_dictionary = compute_properties(
    qc_molecule=qc_mol,
    density=density,
    esp_settings=esp_settings
)
variables_dictionary_bohr = compute_properties_bohr(
    qc_molecule=qc_mol,
    density=density,
    esp_settings=esp_settings
)

/scratch/ipykernel_24327/3908842912.py:27: FutureWarning: Using QCVariable `MULLIKEN_CHARGES` instead of `MULLIKEN CHARGES` is deprecated, and as soon as 1.5 it will stop working

  variables_dictionary["MULLIKEN_CHARGES"] = psi4_wavefunction.variable("MULLIKEN_CHARGES") * unit.e
/scratch/ipykernel_24327/3908842912.py:28: FutureWarning: Using QCVariable `LOWDIN_CHARGES` instead of `LOWDIN CHARGES` is deprecated, and as soon as 1.5 it will stop working

  variables_dictionary["LOWDIN_CHARGES"] = psi4_wavefunction.variable("LOWDIN_CHARGES") * unit.e


In [8]:
variables_dictionary.values()

dict_values([<Quantity([-0.42343836 -0.02856551 -0.61890532  0.08099879  0.43452504 -0.86142776
 -0.71630309  0.02218535 -0.72237869  0.2599009   0.22160784  0.22282849
  0.21847899  0.22778904  0.45508529  0.46648834  0.47547881  0.48469556
  0.17623285  0.15496506  0.46975838], 'elementary_charge')>, <Quantity([-0.30092234 -0.08299242 -0.33591361  0.00447306  0.21622156 -0.37765444
 -0.44551543 -0.08965971 -0.50001035  0.18913101  0.17022335  0.16261253
  0.16351268  0.16576319  0.33788228  0.33875215  0.34629619  0.38586387
  0.14465695  0.12817959  0.37909988], 'elementary_charge')>, <Quantity([[-0.40396124]
 [ 0.15725211]
 [-0.42722573]
 [ 0.02516108]
 [ 0.54580863]
 [-0.71798066]
 [-0.66961351]
 [ 0.15094307]
 [-0.66767662]
 [ 0.18242196]
 [ 0.15165317]
 [ 0.09073137]
 [ 0.0917741 ]
 [ 0.12792078]
 [ 0.41413422]
 [ 0.42417829]
 [ 0.43807062]
 [ 0.49181435]
 [ 0.06992569]
 [ 0.04592019]
 [ 0.47871692]], 'elementary_charge')>, <Quantity([[-0.03332046  0.00545656 -0.00718716]
 [ 0.1

In [9]:
variables_dictionary['MBIS CHARGES']

Magnitude,[[-0.4039612421857906] [0.15725211014772014] [-0.4272257268142511] [0.025161081404444705] [0.545808629307594] [-0.7179806559621535] [-0.6696135101369489] [0.15094307142486071] [-0.6676766246884195] [0.1824219551494599] [0.15165316580455296] [0.09073137474913412] [0.09177409630102076] [0.12792077626461235] [0.414134222724211] [0.42417829200107804] [0.4380706241724684] [0.4918143536650322] [0.06992568566418296] [0.04592018772487958] [0.4787169216653226]]
Units,elementary_charge


In [10]:
variables_dictionary_bohr['MBIS CHARGES']

Magnitude,[[-0.4039612421857308] [0.15725211014797266] [-0.4272257268143906] [0.02516108140444528] [0.5458086293077425] [-0.7179806559622016] [-0.6696135101368873] [0.15094307142495486] [-0.6676766246882914] [0.18242195514938747] [0.15165316580444962] [0.09073137474949973] [0.09177409630040886] [0.12792077626467724] [0.41413422272444367] [0.4241782920011285] [0.43807062417233616] [0.4918143536649385] [0.06992568566392923] [0.04592018772470026] [0.47871692166544794]]
Units,elementary_charge


In [11]:
openff_molecule.assign_partial_charges('am1bcc')

In [12]:
openff_molecule.partial_charges

Magnitude,[-0.1584 0.12039999999999998 -0.3686 0.1401 0.306 -0.8686 -0.5498 0.1344 -0.6618 0.09019999999999999 0.09019999999999999 0.09269999999999999 0.09269999999999999 0.07069999999999999 0.4678 0.4678 0.4678 0.451 0.08219999999999998 0.08219999999999998 0.451]
Units,elementary_charge
